Copyright 2023 The Google Research Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
%git clone https://github.com/google-research/google-research.git
%pip install tensorflow

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from android_in_the_wild import visualization_utils
import tensorflow as tf
import random

In [ ]:
dataset_name = 'google_apps'  #@param ["general", "google_apps", "install", "single", "web_shopping"]

dataset_directories = {
    'general': 'gs://gresearch/android-in-the-wild/general/*',
    'google_apps': 'gs://gresearch/android-in-the-wild/google_apps/*',
    'install': 'gs://gresearch/android-in-the-wild/install/*',
    'single': 'gs://gresearch/android-in-the-wild/single/*',
    'web_shopping': 'gs://gresearch/android-in-the-wild/web_shopping/*',
}

In [ ]:
filenames = tf.io.gfile.glob(dataset_directories[dataset_name])
raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP').as_numpy_iterator()

In [ ]:
def get_episode(dataset):
  """Grabs the first complete episode."""
  episode = []
  episode_id = None
  for d in dataset:
    ex = tf.train.Example()
    ex.ParseFromString(d)
    ep_id = ex.features.feature['episode_id'].bytes_list.value[0].decode('utf-8')
    if episode_id is None:
      episode_id = ep_id
      episode.append(ex)
    elif ep_id == episode_id:
      episode.append(ex)
    else:
      break
  return episode

def get_all_episode(dataset):
  episodes = {}
  episode = []
  episode_id = None
  for d in dataset:
    ex = tf.train.Example()
    ex.ParseFromString(d)
    ep_id = ex.features.feature['episode_id'].bytes_list.value[0].decode('utf-8')
    if episode_id is None:
      episode_id = ep_id
      episode.append(ex)
    elif ep_id == episode_id:
      episode.append(ex)
    else:
      # ep_id != episode_id
      episodes[ep_id] = episode; episode = []
      episode_id = ep_id
      episode.append(ex)
  return episode

def get_all_episodes_num(dataset):
  episodes = {}
  episode = []
  episode_id = None
  cnt = 0
  for d in dataset:
    ex = tf.train.Example()
    ex.ParseFromString(d)
    ep_id = ex.features.feature['episode_id'].bytes_list.value[0].decode('utf-8')
    if episode_id is None:
      episode_id = ep_id; cnt += 1
    elif ep_id == episode_id:
      pass
    else:
      # ep_id != episode_id
      # episodes[ep_id] = episode; episode = []
      episode_id = ep_id; cnt += 1
      # episode.append(ex)
  return cnt

In [ ]:
ep = get_episode(raw_dataset)
visualization_utils.plot_episode(ep, show_annotations=True, show_actions=True)

In [ ]:
# eps = get_all_episode(raw_dataset)
# print(len(eps))

In [ ]:
eps_num = get_all_episodes_num(raw_dataset)